In [ ]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch import optim as optim

In [ ]:
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
class ResBlock(nn.Module):
    def __init__(self, input_ch, output_ch, stride=1, downsample=None):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_ch, out_channels=output_ch, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(output_ch)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=output_ch, out_channels=output_ch, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(output_ch)
        self.downsample = downsample
    
    def forward(self, inputs):
        residual = inputs
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        if self.downsample:
            residual = self.downsample(inputs)
        x += residual
        output = self.relu(x)
        return output

In [ ]:
class CommonBlock(nn.Module):
    def __init__(self, input_ch, output_ch, stride=1, downsample=None):
        super(CommonBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_ch, out_channels=output_ch, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(output_ch)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=output_ch, out_channels=output_ch, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(output_ch)
        self.downsample = downsample
    
    def forward(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        output = self.relu(x)
        return output

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.c1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.layer1 = self.make_layer(block, 16, 3)
        self.layer2 = self.make_layer(block, 32, 3, 2)
        self.layer3 = self.make_layer(block, 64, 3, 2)
        self.avg_pool = nn.AvgPool2d(kernel_size=8)
        self.fc = nn.Linear(64, 10)
    
    def make_layer(self, block, output_ch, num_blocks, stride=1):
        downsample = None
        if (self.in_channels != output_ch) or (stride != 1):
            downsample = nn.Sequential(
                nn.Conv2d(in_channels=self.in_channels, out_channels=output_ch, kernel_size=3, stride=stride, padding=1),
                nn.BatchNorm2d(output_ch)
            )
        layers = []
        layers.append(block(self.in_channels, output_ch, stride, downsample))
        self.in_channels = output_ch
        for i in range(1, num_blocks):
            layers.append(block(output_ch, output_ch))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.c1(x)
        x = self.bn(x)
        x = self.relu(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.c1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()
        self.layer1 = self.make_layer(block, 16, 3)
        self.layer2 = self.make_layer(block, 32, 3, 2)
        self.layer3 = self.make_layer(block, 64, 3, 2)
        self.avg_pool = nn.AvgPool2d(kernel_size=8)
        self.fc = nn.Linear(64, 10)
    
    def make_layer(self, block, output_ch, num_blocks, stride=1):
        downsample = None
        if (self.in_channels != output_ch) or (stride != 1):
            downsample = nn.Sequential(
                nn.Conv2d(in_channels=self.in_channels, out_channels=output_ch, kernel_size=3, stride=stride, padding=1),
                nn.BatchNorm2d(output_ch)
            )
        layers = []
        layers.append(block(self.in_channels, output_ch, stride, downsample))
        self.in_channels = output_ch
        for i in range(1, num_blocks):
            layers.append(block(output_ch, output_ch))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.c1(x)
        x = self.bn(x)
        x = self.relu(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
model = ResNet(CommonBlock).to(device)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
num_params

In [ ]:
BATCH_SIZE = 128
WEIGHT_DECAY = 0.0001
MOMENTUM = 0.9
LEARNING_RATE_BASE = 0.001
EPOCH = 165

learning_rate = LEARNING_RATE_BASE
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=WEIGHT_DECAY,momentum=MOMENTUM)

In [ ]:
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor(),
])
dataset_train = datasets.CIFAR10(root="./data", download=True, train=True, transform=transform)
dataset_test = datasets.CIFAR10(root="./data", train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
num_iteration = 0
loss_data = []
for epoch in range(EPOCH):
    for i, (inputs, labels) in enumerate(train_loader):
        num_iteration += 1
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_data.append(loss.item())

        if num_iteration % 100 == 0:
            print("{}/165 Epoch----{}k/64k iterations: Loss:{:.6f}".format(epoch, num_iteration / 1000, loss.item()))
            torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss_data,
                    }, "checkpoint.pt")
        
        if num_iteration == 32000 or num_iteration == 48000:
            learning_rate = learning_rate / 10
            update_lr(optimizer, learning_rate)

model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for test_inputs, test_labels in test_loader:
        test_inputs = test_inputs.to(device)
        test_labels = test_labels.to(device)
        
        outs = model(test_inputs)
        _, predicted = torch.max(outs.data, 1)
        total += test_labels.size(0)
        correct += (predicted == test_labels).sum().item()
        
    print("Accuracy:{}%".format(100 * correct / total))

# save model
torch.save(model.state_dict(), "cifar10_resnet.ckpt")

In [ ]:
import matplotlib.pyplot as plt

plt.title('Train loss per iteration')
plt.xlabel('Iteration')
plt.ylabel('Train loss')
plt.plot(range(0, len(loss_data[::1000])), loss_data[::1000], "o-")
plt.show()